In [1]:
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

In [14]:
print("----------------Reading the Data-------------------------")
PATH = os.getcwd()
os.chdir('Alphabets/')

X_train = pd.read_csv('train.csv', sep=',', header=None, index_col=False)
X_test = pd.read_csv('test.csv', sep=',', header=None, index_col=False)
np.random.shuffle(X_train.to_numpy())
train_class = X_train[X_train.columns[-1]]
test_actual_class = X_test[X_test.columns[-1]]

X_train = X_train.drop(X_train.columns[-1], axis=1)
X_test = X_test.drop(X_test.columns[-1], axis=1)

print("----------------Data Reading completed-------------------")

os.chdir('../')

X_train = X_train/255
X_test = X_test/255

m = X_train.shape[0] # Number of Training Samples

X_valid = X_train.iloc[(int(0.85*m)):]
valid_class = train_class[(int(0.85*m)):]
X_train = X_train.iloc[0:int(0.85*m)]
train_class = train_class[0:int(0.85*m)]


m = X_train.shape[0] # Number of Training Samples
n = X_train.shape[1] # Number of input features

print("The total number of training samples = {}".format(m))
print("The total number of validation samples = {}".format(X_valid.shape[0]))

print("The number of features = {}".format(n))

----------------Reading the Data-------------------------
----------------Data Reading completed-------------------
The total number of training samples = 11050
The total number of validation samples = 1950
The number of features = 784


In [15]:
#To get the one hot encoding of each label
print("--------Perform 1-hot encoding of class labels------------")

train_class_enc = pd.get_dummies(train_class).to_numpy()
valid_class_enc = pd.get_dummies(valid_class).to_numpy()
test_actual_class_enc = pd.get_dummies(test_actual_class).to_numpy()

--------Perform 1-hot encoding of class labels------------


In [16]:
#Add the intercept term to the data samples both in training and test dataset
X_train = np.hstack((np.ones((m,1)),X_train.to_numpy()))
X_valid = np.hstack((np.ones((X_valid.shape[0],1)), X_valid.to_numpy()))
X_test = np.hstack((np.ones((X_test.shape[0],1)),X_test.to_numpy()))

In [23]:
lr = 0.1
arch_test = [1,5,10,50,100]
arch = [50] #means one hidden layer with 2 perceptrons 
batch_size = 100 # Mini-Batch Size
r = np.max(train_class) + 1 # Default value of the number of classes = 26

In [24]:
#Mini-Batch formation
mini_batch = [(X_train[i:i+batch_size,:], train_class_enc[i:i+batch_size]) for i in range(0, m, batch_size)]
print("The number of mini-batches formed is = {}".format(len(mini_batch)))

The number of mini-batches formed is = 111


In [25]:
#Theta Initialization 
#np.random.seed(1)
def theta_init(arch=50):
    theta = []
    arch = [arch]
    for i in range(len(arch)+1):
        if i == 0:
            dim0=n+1
            dim1=arch[i]
        elif (i == len(arch)):
            dim0=arch[i-1]
            dim1 = r
        else:
            dim0=arch[i-1]
            dim1= arch[i]

        theta.append(2*np.random.random((dim0, dim1))-1)
        #theta.append(np.zeros((dim0, dim1)))
    return theta

In [26]:
def activation(x):
    return 1/(1+np.exp(-x))

In [27]:
def forward_prop(data, theta):
    fm = []
    fm.append(data)
    for l in range(len(theta)):
        fm.append(activation(np.dot(fm[l], theta[l])))
    return fm

In [28]:
def cost_total(X, theta, Y, m):
    fm = forward_prop(X, theta)
    cost = (1/(2*m))*np.sum((Y-fm[-1])**2)
    return cost

In [29]:
def calc_accuracy(data, theta, actual_class):
    pred_class = forward_prop(data, theta)
    test_pred_class = pred_class[-1]
    for i in range(len(test_pred_class)):
        test_pred_class[i][test_pred_class[i] == np.max(test_pred_class[i])] = 1
        test_pred_class[i][test_pred_class[i] != np.max(test_pred_class[i])] = 0


    test_acc = 0
    for i in range(len(actual_class)):
        if (np.array_equal(test_pred_class[i], actual_class[i])):
            test_acc+=1
    test_acc /= data.shape[0]

    #print("The Test Accuracy of the model = {}%".format(test_acc*100))
    return (test_acc*100)

In [31]:
epochs = []
train_accuracy = []
test_accuracy = []
valid_accuracy = []
train_time = []

In [ ]:
arch_test=[100]
for i in range(len(arch_test)):
    theta = theta_init(arch_test[i])
    fm = forward_prop(X_train, theta)
    
    epoch = 0
    start = time.time()
    cost_init = cost_total(X_valid, theta, valid_class_enc, m)

    while(True):
        count = 0

        print("Initial Cost on Val dataset for this epoch {} = {}".format(epoch, cost_init))

        for b in mini_batch:
            X_b = b[0]
            Y_b = b[1]
            fm = forward_prop(X_b, theta)
            delta = [None]*len(fm)

            if (count % 60 == 0):
                print("Error on this batch = "+str(cost_total(X_b, theta, Y_b, batch_size)))
            #Backward Propagation

            for l in range(len(fm)-1, 0, -1):
                if (l == len(fm)-1):
                    delta[l] = ((1/batch_size)*(Y_b - fm[l])*fm[l]*(1-fm[l]))
                else:
                    delta[l]=(np.dot(delta[l+1], theta[l].T)*fm[l]*(1-fm[l]))

            #delta_l1 = np.dot(delta_l2, theta2.T)*l1*(1-l1)

            for t in range(len(theta)):
                theta[t] += lr*np.dot(fm[t].T, delta[t+1]) 

            count+=1
        epoch+=1 #Number of epochs
        cost_final = cost_total(X_valid, theta, valid_class_enc, m)
        print("Cost on val dataset after {} epochs is = {}".format(epoch, cost_final))
        if (abs(cost_final-cost_init) < 1e-07):
            print("cost initial= {} , cost final={} , change in cost= {}".format(cost_init,cost_final, cost_final-cost_init))
            break
        cost_init = cost_final
    epochs.append(epoch)
    train_time.append(time.time()-start)
    train_accuracy.append(calc_accuracy(X_train, theta, train_class_enc))
    valid_accuracy.append(calc_accuracy(X_valid, theta, valid_class_enc))
    test_accuracy.append(calc_accuracy(X_test, theta, test_actual_class_enc))
    print("\n------------------------------------------------------------------------------")
    print("The stats for number of units in the hidden layer = {} are as below:".format(arch_test[i]))
    print("------------------------------------------------------------------------------")
    print("The number of epochs = {}".format(epochs[-1]))
    print("The training time = {}sec".format(train_time[-1]))
    print("The training accuracy is = {}%".format(train_accuracy[-1]))
    print("The test accuracy is = {}%".format(test_accuracy[-1]))
    print("------------------------------------------------------------------------------\n")

Initial Cost on Val dataset for this epoch 0 = 1.2709811606239765
Error on this batch = 7.203002347655447
Error on this batch = 1.0409819188428509
Cost on val dataset after 1 epochs is = 0.095603217978304
Initial Cost on Val dataset for this epoch 1 = 0.095603217978304
Error on this batch = 0.5468827098776943
Error on this batch = 0.5211953983394669
Cost on val dataset after 2 epochs is = 0.08918018980112097
Initial Cost on Val dataset for this epoch 2 = 0.08918018980112097
Error on this batch = 0.510316235169181
Error on this batch = 0.5075694857571176
Cost on val dataset after 3 epochs is = 0.08800755148447219
Initial Cost on Val dataset for this epoch 3 = 0.08800755148447219
Error on this batch = 0.5027989057331503
Error on this batch = 0.5026371991068078
Cost on val dataset after 4 epochs is = 0.08744190907898373
Initial Cost on Val dataset for this epoch 4 = 0.08744190907898373
Error on this batch = 0.49912265967269476
Error on this batch = 0.49970028191740823
Cost on val dataset 

Error on this batch = 0.4503222753265517
Cost on val dataset after 40 epochs is = 0.07915485433701457
Initial Cost on Val dataset for this epoch 40 = 0.07915485433701457
Error on this batch = 0.44038904468016676
Error on this batch = 0.4492313931943811
Cost on val dataset after 41 epochs is = 0.07899606391218275
Initial Cost on Val dataset for this epoch 41 = 0.07899606391218275
Error on this batch = 0.43979562789101695
Error on this batch = 0.44810477671721316
Cost on val dataset after 42 epochs is = 0.07884011143253766
Initial Cost on Val dataset for this epoch 42 = 0.07884011143253766
Error on this batch = 0.4392100190070018
Error on this batch = 0.4469317766021492
Cost on val dataset after 43 epochs is = 0.07868639293676383
Initial Cost on Val dataset for this epoch 43 = 0.07868639293676383
Error on this batch = 0.4386297825682121
Error on this batch = 0.44570020485959333
Cost on val dataset after 44 epochs is = 0.0785341305058985
Initial Cost on Val dataset for this epoch 44 = 0.0

Error on this batch = 0.39908046250989826
Cost on val dataset after 80 epochs is = 0.07201021776441008
Initial Cost on Val dataset for this epoch 80 = 0.07201021776441008
Error on this batch = 0.4095456656296584
Error on this batch = 0.39810389292450793
Cost on val dataset after 81 epochs is = 0.07186295949334127
Initial Cost on Val dataset for this epoch 81 = 0.07186295949334127
Error on this batch = 0.40887836555743334
Error on this batch = 0.39713198206834044
Cost on val dataset after 82 epochs is = 0.07171704616259836
Initial Cost on Val dataset for this epoch 82 = 0.07171704616259836
Error on this batch = 0.40819991329330224
Error on this batch = 0.39616531436440583
Cost on val dataset after 83 epochs is = 0.07157246989800482
Initial Cost on Val dataset for this epoch 83 = 0.07157246989800482
Error on this batch = 0.40751012834282635
Error on this batch = 0.3952045855153184
Cost on val dataset after 84 epochs is = 0.07142921273216513
Initial Cost on Val dataset for this epoch 84 =

Cost on val dataset after 119 epochs is = 0.06502878396244448
Initial Cost on Val dataset for this epoch 119 = 0.06502878396244448
Error on this batch = 0.37137708571859557
Error on this batch = 0.34776113546555854
Cost on val dataset after 120 epochs is = 0.06482003157596108
Initial Cost on Val dataset for this epoch 120 = 0.06482003157596108
Error on this batch = 0.36952266419561014
Error on this batch = 0.3456600076640482
Cost on val dataset after 121 epochs is = 0.06460775294203167
Initial Cost on Val dataset for this epoch 121 = 0.06460775294203167
Error on this batch = 0.3675396541656673
Error on this batch = 0.3434167192662425
Cost on val dataset after 122 epochs is = 0.06438754314031524
Initial Cost on Val dataset for this epoch 122 = 0.06438754314031524
Error on this batch = 0.36548719966819443
Error on this batch = 0.34097940353693273
Cost on val dataset after 123 epochs is = 0.06415723261229972
Initial Cost on Val dataset for this epoch 123 = 0.06415723261229972
Error on thi

Error on this batch = 0.2958800140751189
Cost on val dataset after 159 epochs is = 0.05859754984555925
Initial Cost on Val dataset for this epoch 159 = 0.05859754984555925
Error on this batch = 0.32984797722193565
Error on this batch = 0.29515978447206154
Cost on val dataset after 160 epochs is = 0.05849887094876982
Initial Cost on Val dataset for this epoch 160 = 0.05849887094876982
Error on this batch = 0.32918600105590196
Error on this batch = 0.29444648443168747
Cost on val dataset after 161 epochs is = 0.05840119219328009
Initial Cost on Val dataset for this epoch 161 = 0.05840119219328009
Error on this batch = 0.3285333337525006
Error on this batch = 0.2937396052295144
Cost on val dataset after 162 epochs is = 0.058304481454516735
Initial Cost on Val dataset for this epoch 162 = 0.058304481454516735
Error on this batch = 0.3278904103784436
Error on this batch = 0.29303872562063704
Cost on val dataset after 163 epochs is = 0.05820870397951807
Initial Cost on Val dataset for this e

...

Cost on val dataset after 1173 epochs is = 0.03018813738114852
Initial Cost on Val dataset for this epoch 1173 = 0.03018813738114852
Error on this batch = 0.13830934103090292
Error on this batch = 0.10977317734886136
Cost on val dataset after 1174 epochs is = 0.03017646591426844
Initial Cost on Val dataset for this epoch 1174 = 0.03017646591426844
Error on this batch = 0.13823134827037947
Error on this batch = 0.1097089260843049
Cost on val dataset after 1175 epochs is = 0.03016449442663297
Initial Cost on Val dataset for this epoch 1175 = 0.03016449442663297
Error on this batch = 0.13814956461095407
Error on this batch = 0.10964278023007872
Cost on val dataset after 1176 epochs is = 0.03015221366337037
Initial Cost on Val dataset for this epoch 1176 = 0.03015221366337037
Error on this batch = 0.13806376668906287
Error on this batch = 0.1095746913894095
Cost on val dataset after 1177 epochs is = 0.030139614751032127
Initial Cost on Val dataset for this epoch 1177 = 0.030139614751032127

Cost on val dataset after 1211 epochs is = 0.029549037292174396
Initial Cost on Val dataset for this epoch 1211 = 0.029549037292174396
Error on this batch = 0.1338796792303825
Error on this batch = 0.10661507698397645
Cost on val dataset after 1212 epochs is = 0.02952988038723257
Initial Cost on Val dataset for this epoch 1212 = 0.02952988038723257
Error on this batch = 0.1337798669511032
Error on this batch = 0.10654160685128011
Cost on val dataset after 1213 epochs is = 0.029510800177960008
Initial Cost on Val dataset for this epoch 1213 = 0.029510800177960008
Error on this batch = 0.1336816595899176
Error on this batch = 0.10646960412353561
Cost on val dataset after 1214 epochs is = 0.02949180483055739
Initial Cost on Val dataset for this epoch 1214 = 0.02949180483055739
Error on this batch = 0.13358498244489808
Error on this batch = 0.10639905808394655
Cost on val dataset after 1215 epochs is = 0.029472901646846836
Initial Cost on Val dataset for this epoch 1215 = 0.029472901646846

Error on this batch = 0.10468493574041361
Cost on val dataset after 1249 epochs is = 0.028905387892873807
Initial Cost on Val dataset for this epoch 1249 = 0.028905387892873807
Error on this batch = 0.13077075414114475
Error on this batch = 0.10464903093551216
Cost on val dataset after 1250 epochs is = 0.02889106756058873
Initial Cost on Val dataset for this epoch 1250 = 0.02889106756058873
Error on this batch = 0.13069809587407277
Error on this batch = 0.10461364826852271
Cost on val dataset after 1251 epochs is = 0.028876882139745357
Initial Cost on Val dataset for this epoch 1251 = 0.028876882139745357
Error on this batch = 0.1306254815699576
Error on this batch = 0.10457875951296491
Cost on val dataset after 1252 epochs is = 0.0288628313991932
Initial Cost on Val dataset for this epoch 1252 = 0.0288628313991932
Error on this batch = 0.1305528829821353
Error on this batch = 0.1045443357317362
Cost on val dataset after 1253 epochs is = 0.028848915097595202
Initial Cost on Val dataset

Error on this batch = 0.10344792924288437
Cost on val dataset after 1287 epochs is = 0.02845121121676317
Initial Cost on Val dataset for this epoch 1287 = 0.02845121121676317
Error on this batch = 0.12775152297237152
Error on this batch = 0.10341339734520717
Cost on val dataset after 1288 epochs is = 0.028441512685099046
Initial Cost on Val dataset for this epoch 1288 = 0.028441512685099046
Error on this batch = 0.12765225237891883
Error on this batch = 0.10337858590017313
Cost on val dataset after 1289 epochs is = 0.02843191121876277
Initial Cost on Val dataset for this epoch 1289 = 0.02843191121876277
Error on this batch = 0.12755081874404778
Error on this batch = 0.10334348732626439
Cost on val dataset after 1290 epochs is = 0.02842240482284748
Initial Cost on Val dataset for this epoch 1290 = 0.02842240482284748
Error on this batch = 0.12744707588353157
Error on this batch = 0.10330809364710927
Cost on val dataset after 1291 epochs is = 0.028412991380267295
Initial Cost on Val data

Error on this batch = 0.10168833677807829
Cost on val dataset after 1325 epochs is = 0.028098947713981155
Initial Cost on Val dataset for this epoch 1325 = 0.028098947713981155
Error on this batch = 0.12147957862151242
Error on this batch = 0.1016060803538328
Cost on val dataset after 1326 epochs is = 0.028085208185053237
Initial Cost on Val dataset for this epoch 1326 = 0.028085208185053237
Error on this batch = 0.12126350964117581
Error on this batch = 0.10151853416310658
Cost on val dataset after 1327 epochs is = 0.028070498064548727
Initial Cost on Val dataset for this epoch 1327 = 0.028070498064548727
Error on this batch = 0.12104650540536672
Error on this batch = 0.10142508394890432
Cost on val dataset after 1328 epochs is = 0.02805469927154679
Initial Cost on Val dataset for this epoch 1328 = 0.02805469927154679
Error on this batch = 0.1208281105874786
Error on this batch = 0.1013250578241637
Cost on val dataset after 1329 epochs is = 0.028037693181647796
Initial Cost on Val dat

In [31]:
epochs

[25200]